In [1]:
import pandas as pd
import os, re

In [2]:
pasta_datasets='../datasets/'
pasta_planilha='../planilhas/'

try:
    os.mkdir(pasta_datasets)
    print(f'"Pasta {pasta_datasets} criada') 
except FileExistsError:
    print(f'"Pasta {pasta_datasets} já existe') 

"Pasta ../datasets/ já existe


In [3]:
# Instala pacote necessário para que o Pandas consiga ler os arquivos ODF - Open Document Format for Office Applications
# A partir de março/2011 as planilhas estão nesse formato no site da Receita Federal

# Basta retirar o comentário e executar uma única vez
#!pip install odfpy

In [4]:
# Função utilizada na leitura das planilhas para retirar espaços extras antes e depois da string de interesse
def limpa_espacos(texto):
    return texto.strip()

In [5]:
# cria uma lista com todos os arquivos contidos na pasta_planilha
planilhas = os.listdir(pasta_planilha)

#Cria um dicionário para receber um dataframe por mês
d_df={}

# A partir de maio 2007 as contribuições previdenciárias passaram a ser administradas pela RFB (fusão SRF+SRP)
# A partir de agosto de 2010 a CPPSS passou a ser administrada pela RFB(art. 23, MP nº 497/10).

# Dicionário com a identificação das linhas que contém o total da receita fazendária administrada RFB
d_rotulos = {'RECEITA ADMINISTRADA PELA RFB': 'RECEITA FAZENDARIA TOTAL',
             'RECEITAS ADMINISTRADAS PELA RFB': 'RECEITA FAZENDARIA TOTAL',
             'RECEITA ADMINISTRADA PELA SRF': 'RECEITA FAZENDARIA TOTAL',
             'RECEITA ADMINISTRADA': 'RECEITA FAZENDARIA TOTAL',
             'SUBTOTAL [A]': 'RECEITA FAZENDARIA TOTAL'}

for planilha in planilhas:
    # Verifica apenas arquivos com nome válido e padronizado
    if (re.match('\d{4}-\d{2}-arrecadacao-uf.(xls|ods)',planilha)):
        ano, mes = planilha.split('-',2)[0:2]
                  
        # Nas planilhas os dados de interesse se encontram a partir da linha 11 e nas colunas A até AC
        df = pd.read_excel(pasta_planilha+planilha, header=11, usecols="A:AC", converters={0:limpa_espacos})

        # Realiza preparação dos dados
        
        # Padroniza as diversas denominações atribuídas às colunas com o nome dos tributos
        df.rename(columns={'RECEITA ':'tributo', 'RECEITAS':'tributo',
                           'RECEITAS ':'tributo'}, inplace=True)
        
        # Padroniza o nome da linha que contém a métrica de interesse como 'RECEITA FAZENDARIA TOTAL'
        df['tributo'].replace(d_rotulos, inplace=True)

        # Realiza a transposição do dataframe para tornar os estados nas linhas e a arrecadação dos tributos nas colunas
        df_t = df.transpose().reset_index()
        
        # Utiliza a primeira linha de dados para rótulo das colunas
        df_t.columns = df_t.iloc[0]
        df_t.drop([0,28], inplace=True)
        df_t.rename(columns={'tributo':'estado'}, inplace=True)
        
        # Cria uma coluna com o data (mês) de referência
        df_t['data'] = pd.to_datetime(ano + '-' + mes)
        
        #Agrega as colunas de interesse do dataframe no dicionário com a chave AAAAMM
        d_df[ano+mes] = df_t[['data', 'estado', 'RECEITA FAZENDARIA TOTAL']]

print(f'{len(d_df)} dataframes no dicionário d_df\n')

192 dataframes no dicionário d_df



In [6]:
#Concatena os dataframes de cada mes/ano num único dataframe
df_arrec = pd.concat(d_df, ignore_index=True, sort=False)

# cria um novo índice após concatenar os dataframes
df_arrec.set_index(['data'], inplace=True)

display(df_arrec)

,estado,RECEITA FAZENDARIA TOTAL
data,,
2004-01-01,AC,8540138
2004-01-01,AL,46646041
2004-01-01,AP,10379954
2004-01-01,AM,236111268
2004-01-01,BA,407403246
...,...,...
2019-12-01,RS,3608629696.4
2019-12-01,SC,3580415239.41
2019-12-01,SE,174023110.28


In [7]:
# Simplifica o nome da coluna 'RECEITA FAZENDARIA TOTAL' para 'arrecadacao'
df_arrec.columns=['estado', 'arrecadacao']

# Deixa a escala da arrecadação em R$ 1 milhão
df_arrec['arrecadacao'] = df_arrec['arrecadacao']/1e6

df_arrec.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 5184 entries, 2004-01-01 to 2019-12-01
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   estado       5184 non-null   object
 1   arrecadacao  5184 non-null   object
dtypes: object(2)
memory usage: 121.5+ KB


In [8]:
df_arrec['estado'].unique()

array(['AC', 'AL', 'AP', 'AM', 'BA', 'CE', 'DF', 'ES', 'GO', 'MA', 'MT',
       'MS', 'MG', 'PA', 'PB', 'PR', 'PE', 'PI', 'RJ', 'RN', 'RS', 'RO',
       'RR', 'SC', 'SP', 'SE', 'TO'], dtype=object)

In [9]:
# Exporta em padrão CSV e Excel
df_arrec.to_excel(pasta_planilha+'arrecadacao_uf.xlsx', index=True)
df_arrec.to_csv(pasta_datasets+'arrecadacao_uf.csv', index=True)